# Spike Analysis with Custom Time Ranges

This notebook allows you to create raster plots, electrode schematics, and firing rate analyses with custom time ranges.

In [ ]:
# Import the utility functions
import sys
import os

# Add the path to aligned_utils - handle different working directories
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'Interval_Aligned_Spikes':
    # We're already in the right directory
    pass
else:
    # We're in the parent directory, add the subdirectory to path
    sys.path.append(os.path.join(current_dir, 'Interval_Aligned_Spikes'))

from aligned_utils import (
    load_data, 
    create_raster_plot_custom, 
    create_electrode_schematic, 
    create_firing_heatmap,
    create_summary_statistics,
    run_full_analysis,
    quick_raster,
    get_time_range_info
)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Configure matplotlib for notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Load Data and Get Overview

In [2]:
# Load all data
spikes_df, pico_df, electrode_df = load_data()

# Get time range information
min_time, max_time = get_time_range_info(spikes_df)

# Show basic statistics
create_summary_statistics(spikes_df, pico_df)

Loading data files...


FileNotFoundError: [Errno 2] No such file or directory: 'Interval_Aligned_Spikes/spikes_time_adjusted.csv'

## Create Electrode Schematic

This shows the spatial layout of all electrodes with their numbers.

In [ ]:
# Create electrode schematic
fig_schematic = create_electrode_schematic(electrode_df)
plt.show()

## Create Firing Rate Heatmap

This shows which electrodes are most active across the entire recording.

In [ ]:
# Create firing rate heatmap
fig_heatmap, electrode_rates = create_firing_heatmap(spikes_df, electrode_df)
plt.show()

# Display the electrode firing rates data
print("\nElectrode Firing Rates:")
electrode_rates_display = electrode_rates[['electrode', 'x', 'y', 'firing_rate']].sort_values('firing_rate', ascending=False)
electrode_rates_display.head(10)

## Custom Time Range Raster Plots

Now you can specify custom time ranges for detailed analysis. Modify the `time_start` and `time_end` values below.

In [ ]:
# Example 1: First 30 seconds
time_start = min_time
time_end = min_time + 30

print(f"Creating raster plot for {time_start:.1f}s to {time_end:.1f}s")
fig1 = create_raster_plot_custom(spikes_df, pico_df, time_start, time_end)
plt.show()

In [ ]:
# Example 2: Middle portion of recording
duration = max_time - min_time
time_start = min_time + duration * 0.4  # Start at 40% through recording
time_end = min_time + duration * 0.6    # End at 60% through recording

print(f"Creating raster plot for {time_start:.1f}s to {time_end:.1f}s")
fig2 = create_raster_plot_custom(spikes_df, pico_df, time_start, time_end)
plt.show()

In [ ]:
# Example 3: Custom time range - MODIFY THESE VALUES!
# Set your desired time range here (in seconds)
time_start = 100  # Start time in seconds
time_end = 200    # End time in seconds

# Optional: specify which electrodes to include (leave as None for all)
electrode_subset = None  # e.g., [21, 22, 23] for specific electrodes

print(f"Creating custom raster plot for {time_start}s to {time_end}s")
fig3 = create_raster_plot_custom(spikes_df, pico_df, time_start, time_end, electrode_subset)
plt.show()

## Quick Raster Function

Use this cell to quickly generate raster plots for different time ranges:

In [ ]:
# Quick raster plot function - just specify start and end times
fig_quick = quick_raster(50, 100)  # 50 to 100 seconds
plt.show()

## Save Specific Plots

Use this cell to save plots with custom filenames:

In [ ]:
# Create and save a specific time range plot
time_start = 200
time_end = 300
save_filename = f'custom_raster_{time_start}_{time_end}s.png'

fig_save = create_raster_plot_custom(spikes_df, pico_df, time_start, time_end, save_path=save_filename)
plt.show()

print(f"Plot saved as: {save_filename}")

## Interactive Analysis

Use this section to explore different time ranges interactively:

In [ ]:
# Function to analyze spike activity in a time window
def analyze_time_window(start_time, end_time):
    """Analyze spike activity in a specific time window"""
    window_spikes = spikes_df[
        (spikes_df['time_sec'] >= start_time) & 
        (spikes_df['time_sec'] <= end_time)
    ]
    
    window_pico = pico_df[
        (pico_df['Start_sec'] <= end_time) & 
        (pico_df['End_sec'] >= start_time)
    ]
    
    duration = end_time - start_time
    total_spikes = len(window_spikes)
    firing_rate = total_spikes / duration
    active_electrodes = len(window_spikes['electrode'].unique())
    pico_count = len(window_pico)
    
    print(f"\nTime window analysis ({start_time:.1f}s - {end_time:.1f}s):")
    print(f"Duration: {duration:.1f}s")
    print(f"Total spikes: {total_spikes:,}")
    print(f"Firing rate: {firing_rate:.2f} spikes/second")
    print(f"Active electrodes: {active_electrodes}")
    print(f"Pico intervals: {pico_count}")
    
    return window_spikes, window_pico

# Example usage
window_spikes, window_pico = analyze_time_window(100, 150)

In [ ]:
# Find time windows with highest activity
def find_high_activity_windows(window_size=30, top_n=5):
    """Find time windows with highest spike activity"""
    time_bins = np.arange(min_time, max_time, window_size)
    activities = []
    
    for start in time_bins:
        end = min(start + window_size, max_time)
        window_spikes = spikes_df[
            (spikes_df['time_sec'] >= start) & 
            (spikes_df['time_sec'] < end)
        ]
        rate = len(window_spikes) / window_size
        activities.append((start, end, rate, len(window_spikes)))
    
    # Sort by firing rate
    activities.sort(key=lambda x: x[2], reverse=True)
    
    print(f"\nTop {top_n} most active {window_size}s windows:")
    print("Start\tEnd\tRate(sp/s)\tTotal Spikes")
    print("-" * 40)
    
    for i, (start, end, rate, total) in enumerate(activities[:top_n]):
        print(f"{start:.1f}s\t{end:.1f}s\t{rate:.1f}\t\t{total}")
    
    return activities[:top_n]

# Find most active periods
top_windows = find_high_activity_windows()

In [ ]:
# Plot the most active window
if top_windows:
    most_active = top_windows[0]
    start, end = most_active[0], most_active[1]
    
    print(f"\nPlotting most active window: {start:.1f}s - {end:.1f}s")
    fig_active = create_raster_plot_custom(spikes_df, pico_df, start, end)
    plt.show()